### Setting up the two databases
With using the cassandra database, this file will create a cassandra node with 3 keyspaces. These keyspaces are:


1. **fish_data**

which is data from Barentwatch API

2. **weather_data**
 
with data from Frost API

Here they are only created, data will be added in the following notebooks:

1. 
2. 



#### Connecting to the cassandra database

In [1]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

##### Setting up keyspaces for port 9042

In [2]:

session.execute(
    "CREATE KEYSPACE IF NOT EXISTS fish_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)

session.execute(
    "CREATE KEYSPACE IF NOT EXISTS weather_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)

#### Setting up table for more detailed information about specific locality
Inserting data from 2022

id = 35297

In [3]:
session.set_keyspace('fish_data')

session.execute("DROP TABLE id_35297")

table_creation_query = """
    CREATE TABLE id_35297 (
        datetime TEXT PRIMARY KEY,
        avgAdultFemaleLice FLOAT,
        hasReportedLice BOOLEAN,
        avgMobileLice FLOAT,
        avgStationaryLice FLOAT,
        seaTemperature FLOAT,
    )
    """

session.execute(table_creation_query)

In [4]:
from functions import get_detailed_week_summary
import pandas as pd
from barentswatch.authentication import get_token
token = get_token()
data = []
localityid = 35297
for week in range(1, 53):
    weeksummary = get_detailed_week_summary(token, '2022', str(week), localityid)
    locality_week_data = weeksummary['localityWeek']
    datetime_object = pd.to_datetime('{}-W{}-7'.format(locality_week_data['year'],\
                                    locality_week_data['week']), format='%G-W%V-%u')
    weekly_data = {
    'datetime': datetime_object,
    'avgadultfemalelice': locality_week_data['avgAdultFemaleLice'],
    'hasreportedlice': locality_week_data['hasReportedLice'],
    'avgmobilelice': locality_week_data['avgMobileLice'],
    'avgstationarylice': locality_week_data['avgStationaryLice'],
    'seatemperature': locality_week_data['seaTemperature']
    }
    data.append(weekly_data)
df_35297 = pd.DataFrame(data)
df_35297['datetime'] = df_35297['datetime'].dt.strftime('%Y-%U-%w')

Token request successful


In [5]:
print(locality_week_data)

{'id': 1512288, 'localityNo': 35297, 'year': 2022, 'week': 52, 'hasReportedLice': True, 'hasMechanicalRemoval': False, 'hasBathTreatment': False, 'hasInFeedTreatment': False, 'hasCleanerFishDeployed': False, 'isFallow': False, 'avgAdultFemaleLice': 0.32, 'avgMobileLice': 0.4, 'avgStationaryLice': 0.05, 'seaTemperature': 8.7, 'bathTreatments': [], 'inFeedTreatments': [], 'cleanerFish': None, 'mechanicalRemoval': None, 'timeSinceLastChitinSynthesisInhibitorTreatment': None, 'hasSalmonoids': True, 'isSlaughterHoldingCage': False}


#### Inserting df_35297 into cassandra database


In [6]:
# Prepare the insert statement (replace with your actual table name and columns)
insert_query = session.prepare("""
    INSERT INTO id_35297 (datetime,\
    avgadultfemalelice, hasreportedlice, avgmobilelice, avgstationarylice, seatemperature)
    VALUES (?, ?, ?, ?, ?, ?)
""")

# For each row in the DataFrame, execute the insert query
for index, row in df_35297.iterrows():
    session.execute(insert_query, list(row))

In [7]:
table_35297 = session.execute("SELECT * FROM id_35297")

for row in table_35297:
    print(row)

Row(datetime='2022-51-0', avgadultfemalelice=0.15000000596046448, avgmobilelice=0.3499999940395355, avgstationarylice=0.10000000149011612, hasreportedlice=True, seatemperature=8.0)
Row(datetime='2022-46-0', avgadultfemalelice=0.019999999552965164, avgmobilelice=0.41999998688697815, avgstationarylice=0.05000000074505806, hasreportedlice=True, seatemperature=11.600000381469727)
Row(datetime='2023-01-0', avgadultfemalelice=0.3199999928474426, avgmobilelice=0.4000000059604645, avgstationarylice=0.05000000074505806, hasreportedlice=True, seatemperature=8.699999809265137)
Row(datetime='2022-05-0', avgadultfemalelice=0.09000000357627869, avgmobilelice=0.3400000035762787, avgstationarylice=0.05999999865889549, hasreportedlice=True, seatemperature=7.099999904632568)
Row(datetime='2022-09-0', avgadultfemalelice=0.2199999988079071, avgmobilelice=0.3499999940395355, avgstationarylice=0.0, hasreportedlice=True, seatemperature=7.400000095367432)
Row(datetime='2022-22-0', avgadultfemalelice=0.1800000

#### Creating table for all fish localitties and inserting data for 2022 into it

- Include all variables from Barentwatch 

In [8]:

table_creation_query = """
    CREATE TABLE IF NOT EXISTS locality_data (
        year INT,
        week INT,
        localityNo INT,
        localityWeekId INT PRIMARY KEY,
        name TEXT,
        hasReportedLice BOOLEAN,
        isFallow BOOLEAN,
        avgAdultFemaleLice DOUBLE,
        hasCleanerfishDeployed BOOLEAN,
        hasMechanicalRemoval BOOLEAN,
        hasSubstanceTreatments BOOLEAN,
        hasPd BOOLEAN,
        hasIla BOOLEAN,
        municipalityNo TEXT,
        municipality TEXT,
        lat DOUBLE,
        lon DOUBLE,
        isOnLand BOOLEAN,
        inFilteredSelection BOOLEAN,
        hasSalmonoids BOOLEAN,
        isSlaughterHoldingCage BOOLEAN
    );
"""
session.execute(table_creation_query) # Uncomment to create the table

# Define the INSERT statement
insert_query = """
    INSERT INTO locality_data (year, week, localityNo, localityWeekId, name, hasReportedLice, isFallow, 
                                avgAdultFemaleLice, hasCleanerfishDeployed, hasMechanicalRemoval, 
                                hasSubstanceTreatments, hasPd, hasIla, municipalityNo, municipality, 
                                lat, lon, isOnLand, inFilteredSelection, hasSalmonoids, isSlaughterHoldingCage)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [9]:
from functions import get_week_summary
for week in range(1, 53):
    print(f'Inserting data for week {week}...')
    weeksummary = get_week_summary(token, '2021', str(week))

    for locality in weeksummary['localities']:
        session.execute(
            insert_query,
            (
                weeksummary['year'],
                weeksummary['week'],
                locality['localityNo'],
                locality['localityWeekId'],
                locality['name'],
                locality['hasReportedLice'],
                locality['isFallow'],
                locality['avgAdultFemaleLice'],
                locality['hasCleanerfishDeployed'],
                locality['hasMechanicalRemoval'],
                locality['hasSubstanceTreatments'],
                locality['hasPd'],
                locality['hasIla'],
                locality['municipalityNo'],
                locality['municipality'],
                locality['lat'],
                locality['lon'],
                locality['isOnLand'],
                locality['inFilteredSelection'],
                locality['hasSalmonoids'],
                locality['isSlaughterHoldingCage']
            )
        )

Inserting data for week 1...
Inserting data for week 2...
Inserting data for week 3...
Inserting data for week 4...
Inserting data for week 5...
Inserting data for week 6...
Inserting data for week 7...
Inserting data for week 8...
Inserting data for week 9...
Inserting data for week 10...
Inserting data for week 11...
Inserting data for week 12...
Inserting data for week 13...
Inserting data for week 14...
Inserting data for week 15...
Inserting data for week 16...
Inserting data for week 17...
Inserting data for week 18...
Inserting data for week 19...
Inserting data for week 20...
Inserting data for week 21...
Inserting data for week 22...
Inserting data for week 23...
Inserting data for week 24...
Inserting data for week 25...
Inserting data for week 26...
Inserting data for week 27...
Inserting data for week 28...
Inserting data for week 29...
Inserting data for week 30...
Inserting data for week 31...
Inserting data for week 32...
Inserting data for week 33...
Inserting data for 